In [ ]:
import numpy as np

root = "exhaustive-grid-search/data/rotations/"
rotations = np.load(root + "R_AA_ICO162_S10_removed_duplicates.npy")

### Rotations around the z-axis

In [ ]:
def is_z_axis_rotation(R, tol=1e-6):
    return (
        np.abs(R[0, 2]) < tol
        and np.abs(R[1, 2]) < tol
        and np.abs(R[2, 0]) < tol
        and np.abs(R[2, 1]) < tol
        and np.abs(R[2, 2] - 1) < tol
    )


z_axis_rotations = np.array([R for R in rotations if is_z_axis_rotation(R)])

np.save(root + "R_AA_ICO162_S10_removed_duplicates_z_axis.npy", z_axis_rotations)
print(f"Saved {z_axis_rotations.shape[0]} rotations around the z-axis.")

Saved 44 rotations around the z-axis.


### Rotations with rotation angles below thresholds

In [ ]:
max_angle_deg_x = 5
max_angle_deg_y = 5
max_angle_deg_z = 45


def is_small_rotation(R, max_angle_deg_x, max_angle_deg_y, max_angle_deg_z, eps=1e-6):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < eps

    if not singular:
        x_angle_rad = np.arctan2(R[2, 1], R[2, 2])
        y_angle_rad = np.arctan2(-R[2, 0], sy)
        z_angle_rad = np.arctan2(R[1, 0], R[0, 0])
    else:
        x_angle_rad = np.arctan2(-R[1, 2], R[1, 1])
        y_angle_rad = np.arctan2(-R[2, 0], sy)
        z_angle_rad = 0.0

    x_angle_deg = np.degrees(x_angle_rad)
    y_angle_deg = np.degrees(y_angle_rad)
    z_angle_deg = np.degrees(z_angle_rad)

    return (
        abs(x_angle_deg) < max_angle_deg_x
        and abs(y_angle_deg) < max_angle_deg_y
        and abs(z_angle_deg) < max_angle_deg_z
    )


small_rotations = np.array(
    [
        R
        for R in rotations
        if is_small_rotation(R, max_angle_deg_x, max_angle_deg_y, max_angle_deg_z)
    ]
)

np.save(root + "R_AA_ICO162_S10_small_rotations.npy", small_rotations)
print(
    f"Saved {small_rotations.shape[0]} rotations with rotation angles below thresholds (x: {max_angle_deg_x}°, y: {max_angle_deg_y}°, z: {max_angle_deg_z}°)."
)

Saved 9 rotations with rotation angles below thresholds (x: 5°, y: 5°, z: 45°).


### Rotation space search

In [ ]:
max_angle_deg_x = 5
max_angle_deg_y = 5
max_angle_deg_z = 45
step_deg = 2


def rotation_matrix_from_euler(x_deg, y_deg, z_deg):
    x = np.radians(x_deg)
    y = np.radians(y_deg)
    z = np.radians(z_deg)

    R_x = np.array([[1, 0, 0], [0, np.cos(x), -np.sin(x)], [0, np.sin(x), np.cos(x)]])

    R_y = np.array([[np.cos(y), 0, np.sin(y)], [0, 1, 0], [-np.sin(y), 0, np.cos(y)]])

    R_z = np.array([[np.cos(z), -np.sin(z), 0], [np.sin(z), np.cos(z), 0], [0, 0, 1]])

    return R_z @ R_y @ R_x


x_angles = np.arange(-max_angle_deg_x, max_angle_deg_x + step_deg, step_deg)
y_angles = np.arange(-max_angle_deg_y, max_angle_deg_y + step_deg, step_deg)
z_angles = np.arange(-max_angle_deg_z, max_angle_deg_z + step_deg, step_deg)

rotations = []

for x in x_angles:
    for y in y_angles:
        for z in z_angles:
            R = rotation_matrix_from_euler(x, y, z)
            rotations.append(R)

rotations = np.array(rotations)

np.save(root + "generated_rotations.npy", rotations)
print(f"Saved {rotations.shape[0]} rotation matrices to 'generated_rotations.npy'")

Saved 1656 rotation matrices to 'generated_rotations.npy'
